## IMPORT LIBRARIES

In [1]:
import torch
from torchvision import transforms,models
import torch.nn as nn
from PIL import Image
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm

## Load the trained mode

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "best_bird_model.pth"

In [3]:
num_classes = 200
model = models.resnet50(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [4]:
state_dict = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [5]:
model.to(device)
model.eval()

print(f"ResNet50 model loaded successfully on {device}!")

ResNet50 model loaded successfully on cuda!


## Define Test Transform

In [12]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

## Load Classes

In [6]:
df = pd.read_csv("bird_metadata.csv")

In [7]:
df.head()

,image_id,file_path,class_id,class_name,is_train,full_path,set
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,0,Image_Raw/images/001.Black_footed_Albatross/Bl...,test
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,1,Image_Raw/images/001.Black_footed_Albatross/Bl...,train
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,0,Image_Raw/images/001.Black_footed_Albatross/Bl...,test
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,1,Image_Raw/images/001.Black_footed_Albatross/Bl...,train
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,1,Image_Raw/images/001.Black_footed_Albatross/Bl...,train


In [8]:
test_df = df[df["set"] == "test"].reset_index(drop=True)

In [9]:
test_df.head()

,image_id,file_path,class_id,class_name,is_train,full_path,set
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,0,Image_Raw/images/001.Black_footed_Albatross/Bl...,test
1,3,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,0,Image_Raw/images/001.Black_footed_Albatross/Bl...,test
2,6,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,0,Image_Raw/images/001.Black_footed_Albatross/Bl...,test
3,10,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,0,Image_Raw/images/001.Black_footed_Albatross/Bl...,test
4,12,001.Black_footed_Albatross/Black_Footed_Albatr...,1,001.Black_footed_Albatross,0,Image_Raw/images/001.Black_footed_Albatross/Bl...,test


## Prediction function for a single image

In [10]:
def predict_image(image_path, model, transform):
    image = Image.open(image_path).convert("RGB")
    x = transform(image).unsqueeze(0).to(next(model.parameters()).device)  # add batch dim
    with torch.no_grad():
        outputs = model(x)
        _, predicted_idx = torch.max(outputs, 1)
        predicted_idx = predicted_idx.item()
    return predicted_idx

## Predict for the full test set and map to species name

In [13]:
y_true, y_pred = [], []

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Predicting"):
    path = row["full_path"]          
    true_class_idx = row["class_id"] - 1  
    pred_class_idx = predict_image(path, model, test_transform)
    
    y_true.append(true_class_idx)
    y_pred.append(pred_class_idx)

pred_names = [test_df[test_df['class_id'] == (idx+1)]['class_name'].values[0] for idx in y_pred]

print("Predictions complete for full test set.")


Predicting: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 5794/5794 [01:06<00:00, 86.82it/s]


Predictions complete for full test set.


## Reports

In [14]:
accuracy = accuracy_score(y_true, y_pred)

In [15]:
accuracy

0.7647566448049706